In [1]:
from tqdm import tqdm
import time
from CGWODA import HybridOptimizationAlgorithm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import preprocessing
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.neighbors import KNeighborsClassifier
from zoofs import DragonFlyOptimization

from zoofs import GreyWolfOptimization
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from autogluon.tabular import TabularDataset, TabularPredictor
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score
from zoofs.baseoptimizationalgorithm import BaseOptimizationAlgorithm
from zoofs import DragonFlyOptimization

In [2]:
from sklearn.metrics import log_loss
def objective_function_topass( model, X_train, y_train, X_test, y_test):

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    # Calculate the accuracy of the classifier
    accuracy = accuracy_score(y_test, y_pred)
    # Calculate the classification error rate
    error_rate = 1 - accuracy
    cardinality_R = X_train.shape[1]
    # Calculate the total number of features in C
    total_features_C = 10
    alpha = 0.99
    # Calculate beta
    beta = 1 - alpha
    # Calculate the fitness value
    fitness = error_rate * alpha + beta * (cardinality_R / total_features_C)
    return fitness


In [40]:
from sklearn.neighbors import KNeighborsClassifier

def run_da_optimization(num_iterations, X_train, y_train, X_test, y_test):
    da_results_per_iteration = []
    da_best_feature_lists = []
    da_best_dims = []
    da_best_scores = []

    for _ in tqdm(range(num_iterations), desc="Iterations", unit="iteration"):
        # Replace with your actual algorithm import and instantiation
        algo_object = DragonFlyOptimization(objective_function_topass,
                                            n_iteration=100,
                                            population_size=11,
                                            minimize=True,
                                            logger=None,
                                            
                                           method='random')

        model = KNeighborsClassifier()

        algo_object.fit(model, X_train, y_train, X_test, y_test, verbose=False)

        da_iteration_results = {
            "results_per_iteration": algo_object.best_results_per_iteration,
            "best_dim_list": algo_object.best_dim.tolist(),
            "best_feature_list": algo_object.best_feature_list,
            "best_score": algo_object.best_score
        }
        da_best_feature_list = algo_object.best_feature_list
        da_best_dim = algo_object.best_dim
        da_best_score = algo_object.best_score

        da_results_per_iteration.append(da_iteration_results)
        da_best_feature_lists.append(da_best_feature_list)
        da_best_dims.append(da_best_dim.tolist())
        da_best_scores.append(da_best_score)

    return da_results_per_iteration, da_best_feature_lists, da_best_dims, da_best_scores



In [4]:
def run_cgwoda_optimization(num_iterations, X_train, y_train, X_test, y_test):
    results_per_iteration = []
    best_feature_lists = []
    best_dims = []
    best_scores = []

    for _ in tqdm(range(num_iterations), desc="Iterations", unit="iteration"):
        hybrid_object = HybridOptimizationAlgorithm(
            objective_function=objective_function_topass,
            n_iteration=100,
            population_size=11,
            logger=None,
        )

        knn_classifier = KNeighborsClassifier()

        hybrid_object.fit(knn_classifier, X_train, y_train, X_test, y_test, verbose=False)


        iteration_results = {
            "results_per_iteration": hybrid_object.best_results_per_iteration,
            "best_dim_list": hybrid_object.best_dim.tolist(),
            "best_feature_list": hybrid_object.best_feature_list,
            "best_score": hybrid_object.best_score
        }

        best_feature_list = hybrid_object.best_feature_list
        best_dim = hybrid_object.best_dim
        best_score = hybrid_object.best_score

        results_per_iteration.append(iteration_results)
        best_feature_lists.append(best_feature_list)
        best_dims.append(best_dim.tolist())
        best_scores.append(best_score)

    return results_per_iteration, best_feature_lists, best_dims, best_scores

In [5]:
def preprocess_dataset(dataset):
        # Separate the features and target variable
        if 'id' in dataset.columns:
            dataset = dataset.drop('id', axis=1)
        if 'bmi' in dataset.columns:
            dataset['bmi'].fillna(dataset['bmi'].mean(), inplace=True)
        if 'sex' in dataset.columns:
            sex_mode_val = dataset['sex'].mode().iloc[0]
            dataset['sex'].fillna(sex_mode_val,inplace=True)

        # Perform label encoding on categorical features
        categorical_features = dataset.select_dtypes(include=['object']).columns
        label_encoder = LabelEncoder()
        for feature in categorical_features:
            dataset[feature] = label_encoder.fit_transform(dataset[feature])



        # Normalize the data
        scaler = MinMaxScaler()
        normalized = scaler.fit_transform(dataset)

        # Return the preprocessed dataset
        preprocessed_dataset = pd.DataFrame(dataset, columns=dataset.columns)
        return preprocessed_dataset

## Dataset 1

In [35]:
stroke=pd.read_csv('stroke4.csv')
stroke=preprocess_dataset(stroke)
X = stroke.drop(['stroke'],axis=1)
y = stroke['stroke']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [72]:
num_iterations = 30
results, features, dims, scores = run_cgwoda_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 30/30 [23:43<00:00, 47.44s/iteration]


In [ ]:
scores

In [73]:
features

[['age', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status'],
 ['work_type'],
 ['gender', 'hypertension', 'heart_disease', 'Residence_type'],
 ['age', 'heart_disease', 'ever_married'],
 ['ever_married'],
 ['ever_married', 'work_type'],
 ['age', 'ever_married'],
 ['age', 'heart_disease', 'ever_married'],
 ['gender', 'hypertension', 'work_type'],
 ['age', 'hypertension', 'heart_disease', 'ever_married'],
 ['age', 'ever_married'],
 ['age', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status'],
 ['gender', 'ever_married'],
 ['gender', 'age'],
 ['gender'],
 ['age', 'heart_disease', 'ever_married'],
 ['Residence_type', 'smoking_status'],
 ['gender', 'hypertension', 'heart_disease', 'ever_married', 'smoking_status'],
 ['hypertension', 'heart_disease', 'ever_married', 'smoking_status'],
 ['Residence_type', 'smoking_status'],
 ['gender'],
 ['ever_married'],
 ['ever_married'],
 ['gender', 'hypertension', 'heart_disease', 'ever_married'],
 ['ever_married', 'work_type'],
 ['g

In [78]:
Stroke_data =pd.DataFrame(X[['age', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status']])
Stroke_data['stroke']=y

In [79]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230827_192102\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230827_192102\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   16.61 GB / 255.46 GB (6.5%)
Train Data Rows:    9722
Train Data Columns: 5
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    2313.1 MB
	T

[1000]	valid_set's binary_error: 0.0873587
[2000]	valid_set's binary_error: 0.0688592


	0.9322	 = Validation score   (accuracy)
	2.94s	 = Training   runtime
	0.15s	 = Validation runtime
Fitting model: LightGBM ...
	0.9702	 = Validation score   (accuracy)
	1.81s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9424	 = Validation score   (accuracy)
	1.73s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9424	 = Validation score   (accuracy)
	2.19s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: CatBoost ...
	0.964	 = Validation score   (accuracy)
	13.35s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9342	 = Validation score   (accuracy)
	1.22s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9353	 = Validation score   (accuracy)
	1.1s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.8684	 = Validation score   (accuracy)
	12.19s	 = Training   runtime
	0.04s	

In [ ]:
num_iterations = 30
da_results, da_features, da_dims, da_scores = run_da_optimization(num_iterations, X_train, y_train, X_test, y_test)


Iterations:   3%|▎         | 1/30 [01:12<34:52, 72.14s/iteration] [ 2023-08-21 11:27:32,841 ] 1 individuals went zero
 [ 2023-08-21 11:27:41,057 ] 1 individuals went zero
 [ 2023-08-21 11:27:41,281 ] 2 individuals went zero
Iterations:   7%|▋         | 2/30 [02:22<33:04, 70.87s/iteration] [ 2023-08-21 11:28:26,805 ] 1 individuals went zero
 [ 2023-08-21 11:28:52,640 ] 1 individuals went zero
Iterations:  10%|█         | 3/30 [03:31<31:35, 70.19s/iteration] [ 2023-08-21 11:29:17,312 ] 1 individuals went zero
 [ 2023-08-21 11:29:22,635 ] 1 individuals went zero
 [ 2023-08-21 11:29:26,358 ] 1 individuals went zero
 [ 2023-08-21 11:29:52,146 ] 1 individuals went zero
 [ 2023-08-21 11:29:55,970 ] 1 individuals went zero
 [ 2023-08-21 11:29:57,145 ] 1 individuals went zero
 [ 2023-08-21 11:29:59,969 ] 1 individuals went zero
 [ 2023-08-21 11:30:00,697 ] 1 individuals went zero
Iterations:  13%|█▎        | 4/30 [04:40<30:11, 69.66s/iteration] [ 2023-08-21 11:30:48,193 ] 1 individuals went zer

In [ ]:
da_features

[['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age', 'hypertension', 'ever_married'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age']]

In [ ]:
Stroke_data =pd.DataFrame(X['age'])
Stroke_data['stroke']=y

In [ ]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230821_120536\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230821_120536\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   32.89 GB / 255.46 GB (12.9%)
Train Data Rows:    9722
Train Data Columns: 1
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1791.22 MB


In [ ]:
Stroke_data =pd.DataFrame(X[['age', 'hypertension', 'ever_married']])
Stroke_data['stroke']=y

In [ ]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230821_120837\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230821_120837\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   32.84 GB / 255.46 GB (12.9%)
Train Data Rows:    9722
Train Data Columns: 3
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1551.36 MB


## DataSet 2

In [ ]:
stroke2 = pd.read_csv('stroke/stroke2.csv')

In [ ]:
stroke2=preprocess_dataset(stroke2)
X = stroke2.drop(['stroke'],axis=1)
y = stroke2['stroke']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [ ]:
num_iterations = 30
results, features, dims, scores = run_cgwoda_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations:   7%|▋         | 2/30 [01:28<20:42, 44.39s/iteration] [ 2023-08-21 12:16:47,227 ] 1 individuals went zero
 [ 2023-08-21 12:16:51,298 ] 1 individuals went zero
Iterations:  23%|██▎       | 7/30 [05:25<18:03, 47.12s/iteration] [ 2023-08-21 12:21:07,383 ] 1 individuals went zero
 [ 2023-08-21 12:21:16,765 ] 1 individuals went zero
Iterations:  90%|█████████ | 27/30 [21:04<02:18, 46.25s/iteration] [ 2023-08-21 12:36:32,818 ] 1 individuals went zero
 [ 2023-08-21 12:36:36,832 ] 1 individuals went zero
 [ 2023-08-21 12:36:47,356 ] 1 individuals went zero
 [ 2023-08-21 12:36:49,830 ] 1 individuals went zero
Iterations: 100%|██████████| 30/30 [23:26<00:00, 46.90s/iteration]


In [ ]:
features

[['gender', 'Residence_type'],
 ['age', 'heart_disease', 'ever_married'],
 ['age', 'ever_married', 'work_type'],
 ['gender', 'ever_married', 'work_type', 'Residence_type'],
 ['age', 'work_type'],
 ['gender', 'heart_disease'],
 ['gender'],
 ['hypertension', 'heart_disease', 'ever_married'],
 ['heart_disease', 'avg_glucose_level'],
 ['gender', 'hypertension', 'ever_married'],
 ['gender', 'ever_married'],
 ['age', 'avg_glucose_level', 'bmi'],
 ['gender', 'ever_married'],
 ['gender', 'heart_disease', 'Residence_type'],
 ['gender', 'age', 'ever_married', 'avg_glucose_level', 'bmi'],
 ['gender', 'heart_disease', 'Residence_type'],
 ['age', 'work_type'],
 ['gender', 'ever_married', 'work_type', 'Residence_type'],
 ['age', 'avg_glucose_level', 'bmi'],
 ['gender', 'ever_married', 'work_type', 'Residence_type'],
 ['age', 'heart_disease', 'Residence_type'],
 ['gender', 'heart_disease', 'Residence_type'],
 ['gender', 'heart_disease', 'Residence_type'],
 ['gender', 'heart_disease', 'Residence_type'

In [104]:
Stroke_data =pd.DataFrame(X[['age', 'avg_glucose_level', 'bmi']])
Stroke_data['stroke']=y

In [105]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230827_214049\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230827_214049\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   15.57 GB / 255.46 GB (6.1%)
Train Data Rows:    9466
Train Data Columns: 3
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    2469.71 MB
	

[1000]	valid_set's binary_error: 0.142555
[2000]	valid_set's binary_error: 0.117212
[3000]	valid_set's binary_error: 0.111932


	0.8923	 = Validation score   (accuracy)
	2.89s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_error: 0.0538543


	0.9483	 = Validation score   (accuracy)
	1.13s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9166	 = Validation score   (accuracy)
	1.34s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9134	 = Validation score   (accuracy)
	1.83s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	0.9461	 = Validation score   (accuracy)
	8.13s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9081	 = Validation score   (accuracy)
	0.98s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9113	 = Validation score   (accuracy)
	1.02s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.8036	 = Validation score   (accuracy)
	12.7s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost ...
	0.9187	 = Validation score   (accuracy)
	1.07s	 = Training   runtime
	0.03s	 =

In [ ]:
num_iterations = 30
da_results, da_features, da_dims, da_scores = run_da_optimization(num_iterations, X_train, y_train, X_test, y_test)


Iterations:   0%|          | 0/30 [00:00<?, ?iteration/s] [ 2023-08-21 12:44:02,413 ] 1 individuals went zero
 [ 2023-08-21 12:44:10,368 ] 1 individuals went zero
 [ 2023-08-21 12:44:22,739 ] 1 individuals went zero
Iterations:   7%|▋         | 2/30 [02:14<31:31, 67.57s/iteration] [ 2023-08-21 12:46:28,113 ] 1 individuals went zero
 [ 2023-08-21 12:46:59,709 ] 1 individuals went zero
 [ 2023-08-21 12:47:00,210 ] 1 individuals went zero
 [ 2023-08-21 12:47:05,249 ] 1 individuals went zero
Iterations:  10%|█         | 3/30 [03:25<31:02, 68.99s/iteration] [ 2023-08-21 12:47:10,926 ] 1 individuals went zero
 [ 2023-08-21 12:47:14,544 ] 1 individuals went zero
Iterations:  13%|█▎        | 4/30 [04:33<29:46, 68.69s/iteration] [ 2023-08-21 12:48:15,510 ] 1 individuals went zero
 [ 2023-08-21 12:49:13,047 ] 1 individuals went zero
 [ 2023-08-21 12:49:16,502 ] 1 individuals went zero
 [ 2023-08-21 12:49:17,275 ] 1 individuals went zero
Iterations:  17%|█▋        | 5/30 [05:39<28:12, 67.70s/iter

In [ ]:
da_features

[['age', 'work_type'],
 ['age', 'work_type'],
 ['age'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type']]

In [ ]:
Stroke_data =pd.DataFrame(X[['age', 'work_type']])
Stroke_data['stroke']=y

In [ ]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230821_132030\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230821_132030\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   32.61 GB / 255.46 GB (12.8%)
Train Data Rows:    9466
Train Data Columns: 2
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1610.53 MB


## Dataset 3

In [ ]:
stroke3 = pd.read_csv('stroke/stroke3.csv')

In [ ]:
stroke3=preprocess_dataset(stroke3)
X = stroke3.drop(['stroke'],axis=1)
y = stroke3['stroke']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [ ]:
num_iterations = 30
results, features, dims, scores = run_cgwoda_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations:  20%|██        | 6/30 [54:06<3:38:31, 546.33s/iteration] [ 2023-08-21 14:23:29,664 ] 1 individuals went zero
 [ 2023-08-21 14:28:16,242 ] 1 individuals went zero
 [ 2023-08-21 14:28:55,177 ] 1 individuals went zero
Iterations:  47%|████▋     | 14/30 [2:07:30<2:27:40, 553.77s/iteration] [ 2023-08-21 15:38:59,989 ] 1 individuals went zero
 [ 2023-08-21 15:42:18,262 ] 1 individuals went zero
Iterations:  73%|███████▎  | 22/30 [3:25:40<1:16:12, 571.62s/iteration] [ 2023-08-21 16:58:10,011 ] 1 individuals went zero
 [ 2023-08-21 17:00:31,035 ] 1 individuals went zero
 [ 2023-08-21 17:01:51,730 ] 1 individuals went zero
 [ 2023-08-21 17:02:30,574 ] 1 individuals went zero
 [ 2023-08-21 17:02:59,529 ] 2 individuals went zero
 [ 2023-08-21 17:03:05,459 ] 1 individuals went zero
Iterations: 100%|██████████| 30/30 [19:53:40<00:00, 2387.34s/iteration]    


In [ ]:
features

[['heart_disease', 'Residence_type', 'bmi'],
 ['gender', 'hypertension', 'bmi'],
 ['hypertension', 'heart_disease', 'ever_married', 'smoking_status'],
 ['heart_disease'],
 ['ever_married', 'bmi'],
 ['Residence_type', 'bmi'],
 ['Residence_type', 'bmi'],
 ['hypertension', 'heart_disease', 'ever_married', 'smoking_status'],
 ['hypertension', 'ever_married', 'work_type'],
 ['Residence_type', 'bmi'],
 ['age',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status'],
 ['hypertension', 'bmi'],
 ['age',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status'],
 ['Residence_type', 'bmi'],
 ['work_type'],
 ['age',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status'],
 ['ever_married', 'work_type'],
 ['gender'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'ever_married'],
 ['heart_disease', 'Residence_type'],
 ['bmi'],
 ['heart_disease', 'ever_married', 'bmi'],
 ['gende

In [ ]:
Stroke_data =pd.DataFrame(X[['age',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status']])
Stroke_data['stroke']=y

In [ ]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230822_092619\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230822_092619\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   31.29 GB / 255.46 GB (12.2%)
Train Data Rows:    85234
Train Data Columns: 6
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1548.78 MB

[1000]	valid_set's binary_error: 0.1088
[2000]	valid_set's binary_error: 0.0784
[3000]	valid_set's binary_error: 0.068
[4000]	valid_set's binary_error: 0.0568
[5000]	valid_set's binary_error: 0.0524
[6000]	valid_set's binary_error: 0.0484
[7000]	valid_set's binary_error: 0.0468
[8000]	valid_set's binary_error: 0.046
[9000]	valid_set's binary_error: 0.0432
[10000]	valid_set's binary_error: 0.0408


	0.9592	 = Validation score   (accuracy)
	50.87s	 = Training   runtime
	2.36s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_error: 0.0256
[2000]	valid_set's binary_error: 0.0216


	0.9784	 = Validation score   (accuracy)
	11.48s	 = Training   runtime
	0.18s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9756	 = Validation score   (accuracy)
	17.27s	 = Training   runtime
	0.18s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9748	 = Validation score   (accuracy)
	29.85s	 = Training   runtime
	0.18s	 = Validation runtime
Fitting model: CatBoost ...
	0.9796	 = Validation score   (accuracy)
	110.91s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.972	 = Validation score   (accuracy)
	3.6s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9692	 = Validation score   (accuracy)
	3.6s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.8596	 = Validation score   (accuracy)
	145.33s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: XGBoost ...
	0.9684	 = Validation score   (accuracy)
	10.6s	 = Training   runtime
	0.1

In [ ]:
predictor_combined.leaderboard()

                  model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2     0.9844       3.022124  209.405931                0.004718           1.470291            2       True         14
1              CatBoost     0.9796       0.018949  110.913519                0.018949         110.913519            1       True          7
2              LightGBM     0.9784       0.183970   11.477721                0.183970          11.477721            1       True          4
3         LightGBMLarge     0.9764       0.138153    6.802608                0.138153           6.802608            1       True         13
4      RandomForestGini     0.9756       0.184586   17.273694                0.184586          17.273694            1       True          5
5      RandomForestEntr     0.9748       0.183279   29.845453                0.183279          29.845453            1       True          6
6        ExtraTreesG

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.9844,3.022124,209.405931,0.004718,1.470291,2,True,14
1,CatBoost,0.9796,0.018949,110.913519,0.018949,110.913519,1,True,7
2,LightGBM,0.9784,0.183970,11.477721,0.183970,11.477721,1,True,4
3,LightGBMLarge,0.9764,0.138153,6.802608,0.138153,6.802608,1,True,13
4,RandomForestGini,0.9756,0.184586,17.273694,0.184586,17.273694,1,True,5
5,RandomForestEntr,0.9748,0.183279,29.845453,0.183279,29.845453,1,True,6
6,ExtraTreesGini,0.9720,0.103960,3.604559,0.103960,3.604559,1,True,8
7,ExtraTreesEntr,0.9692,0.075672,3.604416,0.075672,3.604416,1,True,9
8,XGBoost,0.9684,0.129939,10.595365,0.129939,10.595365,1,True,11
9,LightGBMXT,0.9592,2.361809,50.872734,2.361809,50.872734,1,True,3


In [ ]:
num_iterations = 30
da_results, da_features, da_dims, da_scores = run_da_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 30/30 [26:16:20<00:00, 3152.70s/iteration]


In [ ]:
da_features

[['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['gender', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['Residence_type', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi']]

In [ ]:
Stroke_data =pd.DataFrame(X[['hypertension', 'bmi']])
Stroke_data['stroke']=y

In [ ]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230824_135826\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230824_135826\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   20.62 GB / 255.46 GB (8.1%)
Train Data Rows:    85234
Train Data Columns: 2
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1241.89 MB


In [ ]:
Stroke_data =pd.DataFrame(X[['Residence_type', 'bmi']])
Stroke_data['stroke']=y

In [ ]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230824_142837\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230824_142837\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   19.21 GB / 255.46 GB (7.5%)
Train Data Rows:    85234
Train Data Columns: 2
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1916.84 MB


## Dataset 4

In [ ]:
stroke4 = pd.read_csv('stroke/stroke10.csv')

In [ ]:
stroke4=preprocess_dataset(stroke4)
X = stroke4.drop(['stroke'],axis=1)
y = stroke4['stroke']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [ ]:
num_iterations = 30
results, features, dims, scores = run_cgwoda_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 30/30 [1:55:34<00:00, 231.14s/iteration]


In [ ]:
features

[['ever_married', 'work_type', 'avg_glucose_level', 'smoking_status'],
 ['sex', 'ever_married'],
 ['hypertension', 'avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['hypertension', 'heart_disease', 'work_type', 'Residence_type'],
 ['work_type'],
 ['work_type', 'Residence_type', 'smoking_status'],
 ['Residence_type'],
 ['heart_disease'],
 ['Residence_type'],
 ['heart_disease', 'smoking_status'],
 ['avg_glucose_level', 'bmi'],
 ['work_type'],
 ['ever_married', 'Residence_type', 'smoking_status'],
 ['heart_disease', 'smoking_status'],
 ['sex', 'heart_disease', 'Residence_type'],
 ['sex', 'work_type', 'Residence_type', 'smoking_status'],
 ['hypertension'],
 ['avg_glucose_level', 'bmi'],
 ['heart_disease'],
 ['hypertension'],
 ['sex', 'heart_disease', 'Residence_type'],
 ['avg_glucose_level', 'bmi'],
 ['work_type', 'avg_glucose_level', 'bmi'],
 ['heart_disease', 'work_type', 'avg_glucose_level', 'bmi'],
 ['sex', 'ever_married', 'Residence_type'],
 ['heart_disease', 'work_type', 

In [ ]:
Stroke_data =pd.DataFrame(X[['avg_glucose_level', 'bmi']])
Stroke_data['stroke']=y

In [ ]:

predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230825_101829\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230825_101829\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   22.22 GB / 255.46 GB (8.7%)
Train Data Rows:    40920
Train Data Columns: 2
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1640.25 MB


[1000]	valid_set's binary_error: 0.1684
[2000]	valid_set's binary_error: 0.1204
[3000]	valid_set's binary_error: 0.094
[4000]	valid_set's binary_error: 0.0792
[5000]	valid_set's binary_error: 0.0716
[6000]	valid_set's binary_error: 0.062
[7000]	valid_set's binary_error: 0.0572
[8000]	valid_set's binary_error: 0.0512
[9000]	valid_set's binary_error: 0.0464
[10000]	valid_set's binary_error: 0.0424


	0.9576	 = Validation score   (accuracy)
	17.2s	 = Training   runtime
	1.49s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_error: 0.0008


	0.9992	 = Validation score   (accuracy)
	1.97s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: RandomForestGini ...
	1.0	 = Validation score   (accuracy)
	3.73s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: RandomForestEntr ...
	1.0	 = Validation score   (accuracy)
	5.04s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: CatBoost ...
	1.0	 = Validation score   (accuracy)
	31.1s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	1.0	 = Validation score   (accuracy)
	1.7s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	1.0	 = Validation score   (accuracy)
	1.77s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.668	 = Validation score   (accuracy)
	50.74s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: XGBoost ...
	0.9984	 = Validation score   (accuracy)
	1.83s	 = Training   runtime
	0.06s	 = Validation ru

In [ ]:
num_iterations = 30
da_results, da_features, da_dims, da_scores = run_da_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 30/30 [2:44:59<00:00, 329.99s/iteration]


In [ ]:
da_features

[['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['hypertension', 'avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['ever_married', 'work_type', 'avg_glucose_level', 'smoking_status'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['ever_married', 'work_type', 'avg_glucose_l

In [ ]:
Stroke_data =pd.DataFrame(X[['avg_glucose_level', 'bmi']])
Stroke_data['stroke']=y

In [ ]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230825_053305\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230825_053305\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   20.29 GB / 255.46 GB (7.9%)
Train Data Rows:    40920
Train Data Columns: 2
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1159.99 MB


[1000]	valid_set's binary_error: 0.1732
[2000]	valid_set's binary_error: 0.1224
[3000]	valid_set's binary_error: 0.094
[4000]	valid_set's binary_error: 0.0772
[5000]	valid_set's binary_error: 0.0632
[6000]	valid_set's binary_error: 0.0584
[7000]	valid_set's binary_error: 0.056
[8000]	valid_set's binary_error: 0.0524
[9000]	valid_set's binary_error: 0.0484
[10000]	valid_set's binary_error: 0.0424


	0.9576	 = Validation score   (accuracy)
	40.93s	 = Training   runtime
	2.83s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_error: 0.0008


	0.9992	 = Validation score   (accuracy)
	5.26s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: RandomForestGini ...
	1.0	 = Validation score   (accuracy)
	7.84s	 = Training   runtime
	0.22s	 = Validation runtime
Fitting model: RandomForestEntr ...
	1.0	 = Validation score   (accuracy)
	10.26s	 = Training   runtime
	0.21s	 = Validation runtime
Fitting model: CatBoost ...
	1.0	 = Validation score   (accuracy)
	108.85s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	1.0	 = Validation score   (accuracy)
	3.26s	 = Training   runtime
	0.25s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	1.0	 = Validation score   (accuracy)
	3.24s	 = Training   runtime
	0.23s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.6676	 = Validation score   (accuracy)
	127.05s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: XGBoost ...
	0.9976	 = Validation score   (accuracy)
	4.08s	 = Training   runtime
	0.16s	 = Validati

## Dataset 5

In [ ]:
stroke5 = pd.read_csv('stroke/stroke12.csv')

In [ ]:
stroke5=preprocess_dataset(stroke5)
X = stroke5.drop(['stroke'],axis=1)
y = stroke5['stroke']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [ ]:
num_iterations = 30
results, features, dims, scores = run_cgwoda_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|███████████████████████████████████████████████████████████████| 30/30 [01:29<00:00,  3.00s/iteration]


In [ ]:
features

[['hypertension', 'ever_married', 'work_type'],
 ['gender', 'hypertension', 'work_type'],
 ['hypertension', 'ever_married', 'work_type'],
 ['gender',
  'age',
  'hypertension',
  'ever_married',
  'work_type',
  'Residence_type',
  'bmi',
  'smoking_status'],
 ['hypertension', 'work_type'],
 ['gender', 'age', 'hypertension', 'Residence_type', 'bmi', 'smoking_status'],
 ['hypertension', 'ever_married'],
 ['gender', 'age', 'hypertension', 'Residence_type', 'bmi', 'smoking_status'],
 ['heart_disease', 'ever_married', 'work_type'],
 ['gender', 'age', 'hypertension', 'Residence_type', 'bmi', 'smoking_status'],
 ['gender',
  'age',
  'hypertension',
  'work_type',
  'Residence_type',
  'bmi',
  'smoking_status'],
 ['gender', 'hypertension', 'ever_married', 'work_type', 'avg_glucose_level'],
 ['hypertension', 'heart_disease', 'work_type', 'Residence_type'],
 ['work_type', 'Residence_type'],
 ['hypertension', 'work_type'],
 ['hypertension', 'heart_disease', 'ever_married'],
 ['gender', 'age', 

In [ ]:
Stroke_data =pd.DataFrame(X[['gender',
  'age',
  'hypertension',
  'work_type',
  'Residence_type',
  'bmi',
  'smoking_status']])
Stroke_data['stroke']=y

In [ ]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230825_152938\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230825_152938\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   21.48 GB / 255.46 GB (8.4%)
Train Data Rows:    322
Train Data Columns: 7
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1318.4 MB
	Tr

In [ ]:
num_iterations = 30
da_results, da_features, da_dims, da_scores = run_da_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|███████████████████████████████████████████████████████████████| 30/30 [02:09<00:00,  4.32s/iteration]


In [ ]:
da_features

[['gender', 'age', 'hypertension', 'Residence_type', 'bmi', 'smoking_status'],
 ['gender', 'age', 'hypertension', 'Residence_type', 'bmi', 'smoking_status'],
 ['gender', 'age', 'hypertension', 'Residence_type', 'bmi', 'smoking_status'],
 ['gender', 'age', 'hypertension', 'Residence_type', 'bmi', 'smoking_status'],
 ['gender', 'age', 'hypertension', 'Residence_type', 'bmi', 'smoking_status'],
 ['gender', 'age', 'hypertension', 'Residence_type', 'bmi', 'smoking_status'],
 ['gender', 'age', 'hypertension', 'Residence_type', 'bmi', 'smoking_status'],
 ['gender', 'age', 'hypertension', 'Residence_type', 'bmi', 'smoking_status'],
 ['gender', 'age', 'hypertension', 'Residence_type', 'bmi', 'smoking_status'],
 ['gender', 'age', 'hypertension', 'Residence_type', 'bmi', 'smoking_status'],
 ['gender', 'age', 'hypertension', 'Residence_type', 'bmi', 'smoking_status'],
 ['gender', 'age', 'hypertension', 'Residence_type', 'bmi', 'smoking_status'],
 ['gender', 'age', 'hypertension', 'Residence_type',

In [ ]:
Stroke_data =pd.DataFrame(X[['gender',
  'age',
  'hypertension',
  'work_type',
  'Residence_type',
  'bmi',
  'smoking_status']])
Stroke_data['stroke']=y

In [ ]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230825_155837\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230825_155837\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   21.50 GB / 255.46 GB (8.4%)
Train Data Rows:    322
Train Data Columns: 7
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1356.69 MB
	T

## Dataset 6

In [ ]:
stroke6 = pd.read_csv('stroke/stroke 13.csv')

In [ ]:
X = stroke6.drop(['target'],axis=1)
y = stroke6['target']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [ ]:
num_iterations = 30
results, features, dims, scores = run_cgwoda_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|███████████████████████████████████████████████████████████████| 30/30 [02:14<00:00,  4.47s/iteration]


In [ ]:
features

[['sex', 'cp', 'trestbps', 'chol', 'exang', 'slope', 'ca'],
 ['cp', 'restecg', 'exang', 'oldpeak', 'slope', 'ca', 'thal'],
 ['sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'exang', 'slope'],
 ['age', 'chol', 'fbs', 'restecg', 'exang', 'slope', 'thal'],
 ['age', 'sex', 'chol', 'fbs', 'restecg', 'slope', 'ca', 'thal'],
 ['age', 'chol', 'exang'],
 ['sex', 'cp', 'fbs', 'oldpeak', 'slope'],
 ['cp', 'oldpeak', 'slope', 'ca'],
 ['sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'slope', 'ca'],
 ['age', 'sex', 'cp', 'fbs', 'restecg', 'oldpeak', 'ca', 'thal'],
 ['sex', 'cp', 'trestbps', 'chol'],
 ['sex', 'cp', 'trestbps', 'chol', 'exang', 'slope', 'ca'],
 ['sex', 'cp', 'trestbps', 'exang', 'oldpeak', 'slope', 'ca', 'thal'],
 ['sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'ca'],
 ['age', 'sex', 'cp', 'restecg', 'oldpeak', 'ca', 'thal'],
 ['age', 'sex', 'chol', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal'],
 ['sex', 'cp', 'trestbps', 'chol', 'slope', 'ca', 'thal'],
 ['sex', 'cp', 't

In [ ]:
Stroke_data =pd.DataFrame(X[['age', 'sex', 'cp', 'restecg', 'oldpeak', 'slope', 'ca', 'thal']])
Stroke_data['stroke']=y

In [ ]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230826_054656\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230826_054656\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   20.63 GB / 255.46 GB (8.1%)
Train Data Rows:    330
Train Data Columns: 8
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1547.54 MB
	T

In [ ]:
num_iterations = 30
da_results, da_features, da_dims, da_scores = run_da_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|███████████████████████████████████████████████████████████████| 30/30 [04:37<00:00,  9.25s/iteration]


In [ ]:
da_features

[['cp', 'oldpeak', 'slope', 'ca'],
 ['cp', 'exang', 'oldpeak', 'slope', 'ca'],
 ['cp', 'oldpeak', 'slope', 'ca'],
 ['cp', 'oldpeak', 'slope', 'ca'],
 ['sex', 'cp', 'restecg', 'exang', 'slope', 'ca'],
 ['cp', 'fbs', 'oldpeak', 'slope', 'ca'],
 ['cp', 'exang', 'oldpeak', 'slope', 'ca'],
 ['cp', 'oldpeak', 'slope', 'ca'],
 ['cp', 'oldpeak', 'slope', 'ca'],
 ['age', 'sex', 'cp', 'restecg', 'oldpeak', 'ca', 'thal'],
 ['cp', 'oldpeak', 'slope', 'ca'],
 ['sex', 'cp', 'trestbps', 'exang', 'oldpeak', 'slope', 'ca', 'thal'],
 ['sex', 'cp', 'restecg', 'exang', 'slope', 'ca'],
 ['age', 'sex', 'cp', 'restecg', 'oldpeak', 'ca', 'thal'],
 ['sex', 'cp', 'trestbps', 'exang', 'oldpeak', 'slope', 'ca', 'thal'],
 ['age', 'sex', 'cp', 'restecg', 'oldpeak', 'ca', 'thal'],
 ['cp', 'oldpeak', 'slope', 'ca'],
 ['cp', 'restecg', 'oldpeak', 'slope', 'ca'],
 ['sex', 'cp', 'restecg', 'exang', 'slope', 'ca'],
 ['age', 'sex', 'cp', 'restecg', 'oldpeak', 'ca', 'thal'],
 ['age', 'sex', 'cp', 'fbs', 'restecg', 'oldpeak

## Dataset 7

In [16]:
%cd stroke/

/content/drive/MyDrive/Colab Notebooks/stroke


In [17]:
stroke7 = pd.read_csv('stroke14.csv')

In [19]:
stroke7=preprocess_dataset(stroke7)
X = stroke7.drop(['Stroke'],axis=1)
y = stroke7['Stroke']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [20]:
X.shape

(1868, 20)

In [21]:
num_iterations = 30
results, features, dims, scores = run_cgwoda_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 30/30 [09:34<00:00, 19.15s/iteration]


In [22]:
features

[['CholCheck', 'Smoke', 'MentHlth', 'DiffWalk'],
 ['Diabetes_012',
  'HighBP',
  'CholCheck',
  'BMI',
  'Smoke',
  'HeartDiseaseorAttack',
  'PhysicalActivity',
  'Fruits',
  'AlcoholConsump',
  'GenHlth',
  'PhysHlth',
  'DiffWalk',
  'Sex',
  'Education'],
 ['Diabetes_012',
  'HighBP',
  'HighChol',
  'CholCheck',
  'BMI',
  'Smoke',
  'PhysicalActivity',
  'Veggies',
  'PhysHlth',
  'DiffWalk',
  'Sex',
  'Education'],
 ['Diabetes_012',
  'HighBP',
  'BMI',
  'PhysicalActivity',
  'Fruits',
  'Veggies',
  'AnyHealthcare',
  'GenHlth',
  'PhysHlth',
  'DiffWalk',
  'Sex',
  'Education'],
 ['Diabetes_012',
  'HighChol',
  'BMI',
  'Smoke',
  'HeartDiseaseorAttack',
  'PhysicalActivity',
  'Veggies',
  'AnyHealthcare',
  'GenHlth',
  'PhysHlth',
  'DiffWalk',
  'Sex',
  'Education'],
 ['HighChol', 'Smoke', 'Fruits', 'Veggies', 'NoDocbcCost'],
 ['Diabetes_012',
  'HighChol',
  'BMI',
  'Smoke',
  'PhysicalActivity',
  'Fruits',
  'AnyHealthcare',
  'MentHlth',
  'PhysHlth',
  'DiffWalk

In [32]:
Stroke_data =pd.DataFrame(X[ ['Diabetes_012',
  'HighBP',
  'HighChol',
  'CholCheck',
  'BMI',
  'Smoke',
  'HeartDiseaseorAttack',
  'PhysicalActivity',
  'Fruits',
  'Veggies',
  'AlcoholConsump',
  'NoDocbcCost',
  'PhysHlth',
  'DiffWalk',
  'Sex',
  'Age',
  'Education']])
Stroke_data['stroke']=y

In [33]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels/ag-20230826_172715/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230826_172715/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jun 9 10:57:30 UTC 2023
Disk Space Avail:   46.80 GB / 83.96 GB (55.7%)
Train Data Rows:    1868
Train Data Columns: 17
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:       

[1000]	valid_set's binary_error: 0.0748663


	0.9305	 = Validation score   (accuracy)
	0.81s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: LightGBM ...
	0.9358	 = Validation score   (accuracy)
	0.48s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9385	 = Validation score   (accuracy)
	0.88s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9358	 = Validation score   (accuracy)
	1.15s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: CatBoost ...
	0.9519	 = Validation score   (accuracy)
	2.48s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9278	 = Validation score   (accuracy)
	1.02s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9278	 = Validation score   (accuracy)
	1.05s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.9278	 = Validation score   (accuracy)
	2.6s	 = Training   runtime
	0.02s	 = 

In [36]:
num_iterations = 30
da_results, da_features, da_dims, da_scores = run_da_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 30/30 [11:45<00:00, 23.51s/iteration]


In [37]:
da_features

[['Diabetes_012',
  'HighBP',
  'BMI',
  'Smoke',
  'HeartDiseaseorAttack',
  'Veggies',
  'AnyHealthcare',
  'PhysHlth',
  'DiffWalk',
  'Sex',
  'Education'],
 ['Diabetes_012',
  'HighChol',
  'BMI',
  'Fruits',
  'Veggies',
  'AnyHealthcare',
  'PhysHlth',
  'DiffWalk',
  'Sex',
  'Age'],
 ['Diabetes_012',
  'HighBP',
  'HighChol',
  'BMI',
  'Smoke',
  'PhysicalActivity',
  'Fruits',
  'Veggies',
  'AlcoholConsump',
  'AnyHealthcare',
  'PhysHlth',
  'DiffWalk',
  'Sex',
  'Age',
  'Education'],
 ['Diabetes_012',
  'BMI',
  'Smoke',
  'PhysicalActivity',
  'Veggies',
  'AnyHealthcare',
  'PhysHlth',
  'Sex',
  'Education'],
 ['Diabetes_012',
  'HighChol',
  'CholCheck',
  'BMI',
  'Smoke',
  'HeartDiseaseorAttack',
  'PhysicalActivity',
  'Veggies',
  'GenHlth',
  'PhysHlth',
  'DiffWalk',
  'Sex',
  'Education'],
 ['Diabetes_012',
  'HighChol',
  'BMI',
  'Veggies',
  'AlcoholConsump',
  'AnyHealthcare',
  'PhysHlth',
  'DiffWalk',
  'Sex',
  'Age'],
 ['Diabetes_012',
  'BMI',
  '

In [38]:
Stroke_data =pd.DataFrame(X[ ['Diabetes_012',
  'BMI',
  'PhysicalActivity',
  'Fruits',
  'AnyHealthcare',
  'NoDocbcCost',
  'PhysHlth',
  'DiffWalk',
  'Sex',
  'Age',
  'Education']])
Stroke_data['stroke']=y

In [39]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels/ag-20230826_174620/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230826_174620/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jun 9 10:57:30 UTC 2023
Disk Space Avail:   46.74 GB / 83.96 GB (55.7%)
Train Data Rows:    1868
Train Data Columns: 11
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:       

[1000]	valid_set's binary_error: 0.0882353


	0.9171	 = Validation score   (accuracy)
	1.61s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: LightGBM ...
	0.9251	 = Validation score   (accuracy)
	1.1s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9305	 = Validation score   (accuracy)
	1.11s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9332	 = Validation score   (accuracy)
	1.54s	 = Training   runtime
	0.15s	 = Validation runtime
Fitting model: CatBoost ...
	0.9305	 = Validation score   (accuracy)
	2.07s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9144	 = Validation score   (accuracy)
	1.25s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9118	 = Validation score   (accuracy)
	1.46s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.9251	 = Validation score   (accuracy)
	5.41s	 = Training   runtime
	0.03s	 

## Dataset 8

In [8]:
stroke8 = pd.read_csv('stroke/stroke15.csv')

In [9]:
stroke8=preprocess_dataset(stroke8)
X = stroke8.drop(['stroke'],axis=1)
y = stroke8['stroke']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [21]:
X.shape

(9466, 10)

In [24]:
num_iterations = 30
results, features, dims, scores = run_cgwoda_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 30/30 [20:21<00:00, 40.71s/iteration]


In [25]:
features

[['hypertension', 'heart_disease', 'ever_married', 'Residence_type'],
 ['age', 'ever_married', 'Residence_type', 'avg_glucose_level', 'bmi'],
 ['heart_disease'],
 ['age', 'ever_married', 'Residence_type', 'avg_glucose_level', 'bmi'],
 ['gender',
  'age',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi'],
 ['hypertension', 'heart_disease', 'Residence_type'],
 ['age', 'ever_married', 'Residence_type', 'avg_glucose_level', 'bmi'],
 ['gender'],
 ['hypertension', 'ever_married', 'smoking_status'],
 ['age', 'work_type'],
 ['age'],
 ['hypertension',
  'heart_disease',
  'ever_married',
  'work_type',
  'Residence_type'],
 ['hypertension', 'Residence_type'],
 ['age'],
 ['gender', 'hypertension', 'work_type'],
 ['age', 'work_type'],
 ['age'],
 ['Residence_type', 'smoking_status'],
 ['age', 'hypertension', 'ever_married'],
 ['Residence_type', 'smoking_status'],
 ['Residence_type', 'smoking_status'],
 ['age', 'work_type'],
 ['age'],
 ['gender', 'work_type'],
 ['age',
  'hypert

In [22]:
Stroke_data =pd.DataFrame(X[['age',
  'hypertension',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi']])
Stroke_data['stroke']=y

In [23]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230827_112910\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230827_112910\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   19.67 GB / 255.46 GB (7.7%)
Train Data Rows:    9466
Train Data Columns: 6
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1361.81 MB
	

[1000]	valid_set's binary_error: 0.108765
[2000]	valid_set's binary_error: 0.100317
[3000]	valid_set's binary_error: 0.0887012
[4000]	valid_set's binary_error: 0.0855333
[5000]	valid_set's binary_error: 0.0855333


	0.9176	 = Validation score   (accuracy)
	12.59s	 = Training   runtime
	0.67s	 = Validation runtime
Fitting model: LightGBM ...
	0.9483	 = Validation score   (accuracy)
	2.05s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9081	 = Validation score   (accuracy)
	2.37s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9092	 = Validation score   (accuracy)
	3.39s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: CatBoost ...
	0.9504	 = Validation score   (accuracy)
	70.45s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9007	 = Validation score   (accuracy)
	1.7s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9018	 = Validation score   (accuracy)
	1.6s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.8321	 = Validation score   (accuracy)
	20.91s	 = Training   runtime
	0.04

In [23]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230827_112910\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230827_112910\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   19.67 GB / 255.46 GB (7.7%)
Train Data Rows:    9466
Train Data Columns: 6
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1361.81 MB
	

[1000]	valid_set's binary_error: 0.108765
[2000]	valid_set's binary_error: 0.100317
[3000]	valid_set's binary_error: 0.0887012
[4000]	valid_set's binary_error: 0.0855333
[5000]	valid_set's binary_error: 0.0855333


	0.9176	 = Validation score   (accuracy)
	12.59s	 = Training   runtime
	0.67s	 = Validation runtime
Fitting model: LightGBM ...
	0.9483	 = Validation score   (accuracy)
	2.05s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9081	 = Validation score   (accuracy)
	2.37s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9092	 = Validation score   (accuracy)
	3.39s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: CatBoost ...
	0.9504	 = Validation score   (accuracy)
	70.45s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9007	 = Validation score   (accuracy)
	1.7s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9018	 = Validation score   (accuracy)
	1.6s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.8321	 = Validation score   (accuracy)
	20.91s	 = Training   runtime
	0.04

In [27]:
num_iterations = 30
da_results, da_features, da_dims, da_scores = run_da_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 30/30 [32:33<00:00, 65.11s/iteration]


In [28]:
da_features

[['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age', 'heart_disease'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age']]

In [31]:
Stroke_data =pd.DataFrame(X[['age','heart_disease']])
Stroke_data['stroke']=y

In [32]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230827_140859\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230827_140859\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   19.33 GB / 255.46 GB (7.6%)
Train Data Rows:    9466
Train Data Columns: 2
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    2040.84 MB
	

## GreyWolf

In [41]:
def run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test):
    results_per_iteration = []
    best_feature_lists = []
    best_dims = []
    best_scores = []

    for _ in tqdm(range(num_iterations), desc="Iterations", unit="iteration"):
        grey_object = GreyWolfOptimization(
            objective_function=objective_function_topass,
            n_iteration=100,
            population_size=11,
            logger=None,
            method=2
        )

        knn_classifier = KNeighborsClassifier()

        grey_object.fit(knn_classifier, X_train, y_train, X_test, y_test, verbose=False)


        iteration_results = {
            "results_per_iteration": grey_object.best_results_per_iteration,
            "best_dim_list": grey_object.best_dim.tolist(),
            "best_feature_list": grey_object.best_feature_list,
            "best_score": grey_object.best_score
        }

        best_feature_list = grey_object.best_feature_list
        best_dim = grey_object.best_dim
        best_score = grey_object.best_score

        results_per_iteration.append(iteration_results)
        best_feature_lists.append(best_feature_list)
        best_dims.append(best_dim.tolist())
        best_scores.append(best_score)

    return results_per_iteration, best_feature_lists, best_dims, best_scores

In [58]:
num_iterations = 20
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 20/20 [20:20<00:00, 61.00s/iteration]


In [59]:
features

[['age', 'ever_married', 'work_type'],
 ['age', 'heart_disease', 'ever_married'],
 ['age', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status'],
 ['gender', 'age'],
 ['age', 'heart_disease', 'ever_married', 'work_type'],
 ['age', 'ever_married'],
 ['age', 'hypertension', 'heart_disease', 'ever_married'],
 ['age', 'heart_disease'],
 ['age', 'heart_disease', 'ever_married'],
 ['age', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status'],
 ['age', 'hypertension', 'heart_disease', 'ever_married'],
 ['age', 'heart_disease', 'ever_married'],
 ['age', 'heart_disease', 'ever_married'],
 ['age', 'hypertension', 'heart_disease'],
 ['age', 'heart_disease', 'ever_married'],
 ['age', 'heart_disease', 'ever_married', 'work_type'],
 ['age', 'heart_disease', 'ever_married'],
 ['age', 'heart_disease', 'ever_married'],
 ['age', 'heart_disease', 'ever_married'],
 ['gender', 'age']]

In [60]:
num_iterations = 10
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 10/10 [10:09<00:00, 60.97s/iteration]


In [61]:
features

[['age', 'hypertension'],
 ['age', 'work_type'],
 ['age', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status'],
 ['age', 'heart_disease', 'ever_married'],
 ['age', 'work_type'],
 ['age', 'hypertension', 'heart_disease', 'ever_married'],
 ['age', 'heart_disease', 'ever_married'],
 ['gender', 'age'],
 ['age', 'work_type'],
 ['age', 'hypertension', 'heart_disease', 'ever_married']]

In [64]:
Stroke_data =pd.DataFrame(X[['age', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status']])
Stroke_data['stroke']=y

In [65]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230827_184445\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230827_184445\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   17.62 GB / 255.46 GB (6.9%)
Train Data Rows:    9722
Train Data Columns: 5
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    2300.94 MB
	

[1000]	valid_set's binary_error: 0.0873587
[2000]	valid_set's binary_error: 0.0688592


	0.9322	 = Validation score   (accuracy)
	2.95s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: LightGBM ...
	0.9702	 = Validation score   (accuracy)
	1.6s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9424	 = Validation score   (accuracy)
	1.65s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9424	 = Validation score   (accuracy)
	2.13s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: CatBoost ...
	0.964	 = Validation score   (accuracy)
	15.76s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9342	 = Validation score   (accuracy)
	1.09s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9353	 = Validation score   (accuracy)
	1.08s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.8684	 = Validation score   (accuracy)
	12.51s	 = Training   runtime
	0.03s	

## Dataset2

In [80]:
stroke2 = pd.read_csv('stroke/stroke2.csv')

In [81]:
stroke2=preprocess_dataset(stroke2)
X = stroke2.drop(['stroke'],axis=1)
y = stroke2['stroke']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [89]:
num_iterations = 10
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 10/10 [09:52<00:00, 59.30s/iteration]


In [90]:
features

[['age', 'heart_disease', 'ever_married'],
 ['age', 'work_type'],
 ['age', 'hypertension', 'heart_disease'],
 ['age', 'ever_married', 'work_type'],
 ['age', 'hypertension', 'avg_glucose_level', 'bmi'],
 ['age', 'ever_married', 'avg_glucose_level', 'bmi'],
 ['age', 'avg_glucose_level', 'bmi'],
 ['age'],
 ['age', 'ever_married', 'avg_glucose_level', 'bmi'],
 ['age', 'avg_glucose_level', 'bmi']]

In [92]:
num_iterations = 10
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 10/10 [10:00<00:00, 60.00s/iteration]


In [93]:
features

[['age', 'avg_glucose_level', 'bmi'],
 ['age'],
 ['age', 'ever_married', 'work_type'],
 ['age', 'ever_married', 'work_type'],
 ['age', 'avg_glucose_level', 'bmi'],
 ['age', 'heart_disease', 'Residence_type'],
 ['age', 'ever_married', 'avg_glucose_level', 'bmi'],
 ['age', 'ever_married', 'avg_glucose_level', 'bmi'],
 ['age', 'hypertension', 'heart_disease'],
 ['age', 'heart_disease', 'ever_married']]

In [87]:
num_iterations = 10
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 10/10 [09:51<00:00, 59.11s/iteration]


In [88]:
features

[['age', 'heart_disease', 'ever_married'],
 ['age', 'avg_glucose_level', 'bmi'],
 ['age', 'work_type'],
 ['age', 'hypertension', 'heart_disease'],
 ['age', 'work_type'],
 ['age', 'ever_married', 'work_type'],
 ['age', 'hypertension', 'heart_disease'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'ever_married', 'work_type']]

In [94]:
Stroke_data =pd.DataFrame(X[['age', 'ever_married', 'avg_glucose_level', 'bmi']])
Stroke_data['stroke']=y

In [95]:
predictor_combined = TabularPredictor(
    label="stroke",

).fit(
    train_data=Stroke_data,

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230827_212932\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230827_212932\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   16.38 GB / 255.46 GB (6.4%)
Train Data Rows:    9466
Train Data Columns: 4
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    2974.99 MB
	

[1000]	valid_set's binary_error: 0.139388
[2000]	valid_set's binary_error: 0.116156
[3000]	valid_set's binary_error: 0.10982
[4000]	valid_set's binary_error: 0.107709
[5000]	valid_set's binary_error: 0.101373
[6000]	valid_set's binary_error: 0.0992608
[7000]	valid_set's binary_error: 0.0982049
[8000]	valid_set's binary_error: 0.0918691
[9000]	valid_set's binary_error: 0.0887012
[10000]	valid_set's binary_error: 0.0834213


	0.9187	 = Validation score   (accuracy)
	9.67s	 = Training   runtime
	0.82s	 = Validation runtime
Fitting model: LightGBM ...
	0.9472	 = Validation score   (accuracy)
	1.01s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9229	 = Validation score   (accuracy)
	1.67s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9219	 = Validation score   (accuracy)
	2.45s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: CatBoost ...
	0.9514	 = Validation score   (accuracy)
	22.31s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9197	 = Validation score   (accuracy)
	1.06s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9197	 = Validation score   (accuracy)
	1.02s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.8141	 = Validation score   (accuracy)
	12.05s	 = Training   runtime
	0.0

## Dataset3

In [108]:
stroke3 = pd.read_csv('stroke/stroke3.csv')

In [109]:
stroke3=preprocess_dataset(stroke3)
X = stroke3.drop(['stroke'],axis=1)
y = stroke3['stroke']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [110]:
num_iterations = 10
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 10/10 [1:51:17<00:00, 667.74s/iteration]


In [111]:
features

[['bmi'],
 ['hypertension', 'Residence_type', 'bmi'],
 ['hypertension', 'Residence_type', 'bmi'],
 ['heart_disease', 'Residence_type', 'bmi'],
 ['gender', 'hypertension', 'bmi'],
 ['Residence_type', 'bmi'],
 ['age',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status'],
 ['ever_married', 'bmi'],
 ['age',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status'],
 ['age',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status']]

In [112]:
num_iterations = 10
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 10/10 [1:52:25<00:00, 674.52s/iteration]


In [113]:
features

[['age',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status'],
 ['bmi'],
 ['gender', 'ever_married', 'bmi'],
 ['Residence_type', 'bmi'],
 ['age',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status'],
 ['gender', 'hypertension', 'bmi'],
 ['heart_disease', 'Residence_type', 'bmi'],
 ['gender', 'hypertension', 'bmi'],
 ['bmi'],
 ['ever_married', 'Residence_type', 'bmi']]

In [114]:
num_iterations = 10
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 10/10 [1:51:31<00:00, 669.15s/iteration]


In [115]:
features

[['Residence_type', 'bmi'],
 ['heart_disease', 'Residence_type', 'bmi'],
 ['Residence_type', 'bmi'],
 ['heart_disease', 'Residence_type', 'bmi'],
 ['age',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status'],
 ['gender', 'hypertension', 'bmi'],
 ['ever_married', 'Residence_type', 'bmi'],
 ['gender', 'hypertension', 'bmi'],
 ['hypertension', 'Residence_type', 'bmi'],
 ['gender', 'bmi']]

## Dataset4

In [116]:
stroke4 = pd.read_csv('stroke/stroke10.csv')

In [117]:
stroke4=preprocess_dataset(stroke4)
X = stroke4.drop(['stroke'],axis=1)
y = stroke4['stroke']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [119]:
num_iterations = 10
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 10/10 [51:16<00:00, 307.62s/iteration]


In [120]:
features

[['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['hypertension', 'avg_glucose_level', 'bmi'],
 ['heart_disease', 'avg_glucose_level', 'bmi'],
 ['heart_disease', 'avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['heart_disease', 'avg_glucose_level', 'bmi'],
 ['heart_disease', 'avg_glucose_level', 'bmi'],
 ['heart_disease', 'avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi']]

In [121]:
num_iterations = 10
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 10/10 [49:09<00:00, 294.92s/iteration]


In [122]:
features

[['work_type', 'avg_glucose_level', 'bmi'],
 ['work_type', 'avg_glucose_level', 'bmi'],
 ['hypertension', 'avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['hypertension', 'avg_glucose_level', 'bmi'],
 ['hypertension', 'avg_glucose_level', 'bmi'],
 ['ever_married', 'work_type', 'avg_glucose_level', 'smoking_status'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['hypertension', 'avg_glucose_level', 'bmi']]

In [126]:
num_iterations = 10
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 10/10 [51:43<00:00, 310.38s/iteration]


In [127]:
features

[['hypertension', 'avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['heart_disease', 'avg_glucose_level', 'bmi'],
 ['ever_married', 'work_type', 'avg_glucose_level', 'smoking_status'],
 ['hypertension', 'avg_glucose_level', 'bmi'],
 ['hypertension', 'avg_glucose_level', 'bmi'],
 ['work_type', 'Residence_type', 'avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['avg_glucose_level', 'bmi'],
 ['ever_married', 'work_type', 'avg_glucose_level', 'bmi']]

## Dataset5

In [128]:
stroke5 = pd.read_csv('stroke/stroke12.csv')

In [129]:
stroke5=preprocess_dataset(stroke5)
X = stroke5.drop(['stroke'],axis=1)
y = stroke5['stroke']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [130]:
num_iterations = 20
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)


Iterations: 100%|██████████| 20/20 [01:29<00:00,  4.47s/iteration]


In [131]:
features

[['age', 'heart_disease', 'Residence_type', 'smoking_status'],
 ['gender', 'age', 'heart_disease', 'bmi', 'smoking_status'],
 ['ever_married', 'work_type', 'bmi', 'smoking_status'],
 ['ever_married', 'work_type', 'bmi', 'smoking_status'],
 ['ever_married', 'work_type', 'bmi', 'smoking_status'],
 ['age', 'heart_disease', 'Residence_type', 'smoking_status'],
 ['ever_married', 'work_type', 'bmi', 'smoking_status'],
 ['age', 'heart_disease', 'Residence_type', 'smoking_status'],
 ['gender', 'age', 'heart_disease', 'bmi', 'smoking_status'],
 ['age', 'heart_disease', 'Residence_type', 'smoking_status'],
 ['age', 'heart_disease', 'Residence_type', 'smoking_status'],
 ['ever_married', 'work_type', 'bmi', 'smoking_status'],
 ['ever_married', 'work_type', 'bmi', 'smoking_status'],
 ['age', 'heart_disease', 'Residence_type', 'smoking_status'],
 ['gender', 'age', 'heart_disease', 'bmi', 'smoking_status'],
 ['gender', 'age', 'heart_disease', 'bmi', 'smoking_status'],
 ['ever_married', 'work_type', '

In [132]:
num_iterations = 10
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 10/10 [00:43<00:00,  4.30s/iteration]


In [133]:
features

[['gender', 'age', 'heart_disease', 'bmi', 'smoking_status'],
 ['age',
  'hypertension',
  'heart_disease',
  'Residence_type',
  'bmi',
  'smoking_status'],
 ['ever_married', 'work_type', 'bmi', 'smoking_status'],
 ['ever_married', 'work_type', 'bmi', 'smoking_status'],
 ['gender', 'age', 'heart_disease', 'bmi', 'smoking_status'],
 ['age', 'heart_disease', 'Residence_type', 'smoking_status'],
 ['ever_married', 'work_type', 'bmi', 'smoking_status'],
 ['age', 'heart_disease', 'Residence_type', 'smoking_status'],
 ['age', 'heart_disease', 'Residence_type', 'smoking_status'],
 ['gender', 'age', 'heart_disease', 'bmi', 'smoking_status']]

## Dataset6

In [134]:
stroke6 = pd.read_csv('stroke/stroke 13.csv')

In [135]:
X = stroke6.drop(['target'],axis=1)
y = stroke6['target']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [136]:
num_iterations = 20
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 20/20 [02:36<00:00,  7.82s/iteration]


In [137]:
features

[['cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal'],
 ['age', 'sex', 'cp', 'restecg', 'oldpeak', 'slope', 'ca', 'thal'],
 ['age', 'sex', 'cp', 'fbs', 'restecg', 'oldpeak', 'slope', 'ca', 'thal'],
 ['age', 'sex', 'cp', 'restecg', 'oldpeak', 'slope', 'ca', 'thal'],
 ['age', 'sex', 'cp', 'fbs', 'restecg', 'oldpeak', 'ca', 'thal'],
 ['cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal'],
 ['age', 'sex', 'cp', 'restecg', 'oldpeak', 'slope', 'ca', 'thal'],
 ['cp', 'fbs', 'restecg', 'slope', 'ca', 'thal'],
 ['cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal'],
 ['sex', 'cp', 'restecg', 'oldpeak', 'slope', 'ca', 'thal'],
 ['age', 'sex', 'cp', 'restecg', 'oldpeak', 'ca', 'thal'],
 ['sex', 'cp', 'restecg', 'exang', 'slope', 'ca', 'thal'],
 ['age', 'sex', 'cp', 'restecg', 'oldpeak', 'slope', 'ca', 'thal'],
 ['cp', 'restecg', 'exang', 'slope', 'ca', 'thal'],
 ['age', 'sex', 'cp', 'restecg', 'oldpeak', 'ca', 'thal'],
 ['age', 'sex', 'cp', 'restecg', 'oldpeak', 'ca', 'thal'],
 ['age', 's

In [138]:
num_iterations = 10
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 10/10 [01:11<00:00,  7.15s/iteration]


In [139]:
features

[['age', 'sex', 'cp', 'fbs', 'restecg', 'oldpeak', 'slope', 'ca', 'thal'],
 ['age', 'sex', 'cp', 'restecg', 'oldpeak', 'ca', 'thal'],
 ['fbs', 'exang', 'oldpeak', 'slope', 'ca', 'thal'],
 ['cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal'],
 ['cp', 'fbs', 'restecg', 'slope', 'ca', 'thal'],
 ['age', 'sex', 'cp', 'restecg', 'oldpeak', 'ca', 'thal'],
 ['age', 'sex', 'cp', 'restecg', 'oldpeak', 'ca', 'thal'],
 ['cp', 'restecg', 'exang', 'slope', 'ca', 'thal'],
 ['age', 'sex', 'cp', 'restecg', 'oldpeak', 'slope', 'ca', 'thal'],
 ['cp', 'restecg', 'exang', 'slope', 'ca', 'thal']]

## Dataset7

In [141]:
stroke7 = pd.read_csv('stroke/stroke14.csv')[:1000]

In [142]:
stroke7=preprocess_dataset(stroke7)
X = stroke7.drop(['Stroke'],axis=1)
y = stroke7['Stroke']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [143]:
num_iterations = 20
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 20/20 [11:38<00:00, 34.94s/iteration]


In [144]:
features

[['Diabetes_012',
  'HighBP',
  'HighChol',
  'CholCheck',
  'BMI',
  'PhysicalActivity',
  'Fruits',
  'Veggies',
  'AnyHealthcare',
  'NoDocbcCost',
  'GenHlth',
  'PhysHlth',
  'Sex',
  'Education'],
 ['Diabetes_012',
  'CholCheck',
  'BMI',
  'Smoke',
  'PhysicalActivity',
  'Veggies',
  'AlcoholConsump',
  'AnyHealthcare',
  'NoDocbcCost',
  'PhysHlth',
  'DiffWalk',
  'Education'],
 ['Diabetes_012',
  'CholCheck',
  'BMI',
  'Smoke',
  'PhysicalActivity',
  'Fruits',
  'Veggies',
  'AlcoholConsump',
  'AnyHealthcare',
  'NoDocbcCost',
  'GenHlth',
  'PhysHlth',
  'Education'],
 ['Diabetes_012',
  'HighBP',
  'CholCheck',
  'BMI',
  'Smoke',
  'PhysicalActivity',
  'Veggies',
  'AnyHealthcare',
  'PhysHlth',
  'DiffWalk',
  'Sex',
  'Education'],
 ['Diabetes_012',
  'HighBP',
  'BMI',
  'Smoke',
  'HeartDiseaseorAttack',
  'PhysicalActivity',
  'Fruits',
  'Veggies',
  'AnyHealthcare',
  'PhysHlth',
  'DiffWalk',
  'Education'],
 ['Diabetes_012',
  'HighBP',
  'CholCheck',
  'BMI'

In [145]:
num_iterations = 10
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 10/10 [05:38<00:00, 33.85s/iteration]


In [146]:
features

[['Diabetes_012',
  'HighBP',
  'CholCheck',
  'BMI',
  'Smoke',
  'HeartDiseaseorAttack',
  'PhysicalActivity',
  'Fruits',
  'Veggies',
  'AlcoholConsump',
  'AnyHealthcare',
  'PhysHlth',
  'DiffWalk',
  'Sex',
  'Education'],
 ['Diabetes_012',
  'HighBP',
  'BMI',
  'Smoke',
  'PhysicalActivity',
  'Fruits',
  'Veggies',
  'AlcoholConsump',
  'GenHlth',
  'PhysHlth',
  'DiffWalk',
  'Sex',
  'Education'],
 ['Diabetes_012',
  'HighBP',
  'CholCheck',
  'BMI',
  'PhysicalActivity',
  'Veggies',
  'AlcoholConsump',
  'AnyHealthcare',
  'PhysHlth',
  'DiffWalk',
  'Education'],
 ['Diabetes_012',
  'CholCheck',
  'BMI',
  'Smoke',
  'PhysicalActivity',
  'Veggies',
  'AlcoholConsump',
  'AnyHealthcare',
  'PhysHlth',
  'DiffWalk',
  'Education'],
 ['Diabetes_012',
  'HighBP',
  'HighChol',
  'CholCheck',
  'HeartDiseaseorAttack',
  'PhysicalActivity',
  'Fruits',
  'Veggies',
  'AlcoholConsump',
  'AnyHealthcare',
  'NoDocbcCost',
  'GenHlth',
  'PhysHlth',
  'DiffWalk',
  'Sex',
  'Age

## Dataset8

In [147]:
stroke8 = pd.read_csv('stroke/stroke15.csv')

In [148]:
stroke8=preprocess_dataset(stroke8)
X = stroke8.drop(['stroke'],axis=1)
y = stroke8['stroke']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [149]:
num_iterations = 20
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 20/20 [20:10<00:00, 60.52s/iteration]


In [150]:
features

[['gender', 'age', 'ever_married', 'avg_glucose_level', 'bmi'],
 ['age', 'ever_married', 'Residence_type', 'avg_glucose_level', 'bmi'],
 ['gender', 'age', 'ever_married', 'avg_glucose_level', 'bmi'],
 ['gender', 'age', 'ever_married', 'avg_glucose_level', 'bmi'],
 ['gender', 'age', 'ever_married', 'avg_glucose_level', 'bmi'],
 ['age', 'ever_married', 'Residence_type', 'avg_glucose_level', 'bmi'],
 ['age'],
 ['age', 'ever_married', 'Residence_type', 'avg_glucose_level', 'bmi'],
 ['age', 'ever_married', 'Residence_type', 'avg_glucose_level', 'bmi'],
 ['age', 'heart_disease'],
 ['age', 'ever_married'],
 ['gender', 'age', 'ever_married', 'avg_glucose_level', 'bmi'],
 ['gender', 'age', 'ever_married', 'avg_glucose_level', 'bmi'],
 ['age', 'ever_married', 'Residence_type', 'avg_glucose_level', 'bmi'],
 ['age', 'ever_married', 'Residence_type', 'avg_glucose_level', 'bmi'],
 ['age',
  'hypertension',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi'],
 ['age', 'ever_married'

In [152]:
num_iterations = 10
results, features, dims, scores = run_gwo_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 10/10 [10:17<00:00, 61.73s/iteration]


In [153]:
features

[['age',
  'heart_disease',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi'],
 ['age', 'ever_married', 'Residence_type', 'avg_glucose_level', 'bmi'],
 ['gender', 'age', 'ever_married', 'avg_glucose_level', 'bmi'],
 ['age', 'ever_married', 'Residence_type', 'avg_glucose_level', 'bmi'],
 ['age', 'ever_married', 'Residence_type', 'avg_glucose_level', 'bmi'],
 ['age', 'ever_married', 'Residence_type', 'avg_glucose_level', 'bmi'],
 ['age', 'ever_married', 'Residence_type', 'avg_glucose_level', 'bmi'],
 ['age', 'ever_married', 'Residence_type', 'avg_glucose_level', 'bmi'],
 ['age', 'ever_married', 'Residence_type', 'avg_glucose_level', 'bmi'],
 ['age', 'ever_married', 'Residence_type', 'avg_glucose_level', 'bmi']]

## Particle Swarm Optimization

In [159]:
from zoofs import ParticleSwarmOptimization
def run_pso_optimization(num_iterations, X_train, y_train, X_test, y_test):
    results_per_iteration = []
    best_feature_lists = []
    best_dims = []
    best_scores = []

    for _ in tqdm(range(num_iterations), desc="Iterations", unit="iteration"):
        pso_object = ParticleSwarmOptimization(
            objective_function=objective_function_topass,
            n_iteration=100,
            population_size=11,
            logger=None,
        )

        knn_classifier = KNeighborsClassifier()

        pso_object.fit(knn_classifier, X_train, y_train, X_test, y_test, verbose=False)


        iteration_results = {
            "results_per_iteration": pso_object.best_results_per_iteration,
            "best_dim_list": pso_object.best_dim.tolist(),
            "best_feature_list": pso_object.best_feature_list,
            "best_score": pso_object.best_score
        }

        best_feature_list = pso_object.best_feature_list
        best_dim = pso_object.best_dim
        best_score = pso_object.best_score

        results_per_iteration.append(iteration_results)
        best_feature_lists.append(best_feature_list)
        best_dims.append(best_dim.tolist())
        best_scores.append(best_score)

    return results_per_iteration, best_feature_lists, best_dims, best_scores

In [155]:
stroke=pd.read_csv('stroke4.csv')
stroke=preprocess_dataset(stroke)
X = stroke.drop(['stroke'],axis=1)
y = stroke['stroke']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [163]:
num_iterations = 10
results, features, dims, scores = run_pso_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 10/10 [05:52<00:00, 35.24s/iteration]


In [164]:
features

[['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age']]

In [165]:
num_iterations = 10
results, features, dims, scores = run_pso_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 10/10 [05:09<00:00, 30.98s/iteration]


In [166]:
features

[['age'],
 ['age'],
 ['age', 'hypertension'],
 ['gender',
  'age',
  'ever_married',
  'avg_glucose_level',
  'bmi',
  'smoking_status'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['gender',
  'age',
  'ever_married',
  'avg_glucose_level',
  'bmi',
  'smoking_status']]

In [167]:
num_iterations = 10
results, features, dims, scores = run_pso_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 10/10 [05:45<00:00, 34.58s/iteration]


In [168]:
features

[['age'],
 ['age'],
 ['gender', 'age', 'ever_married'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age']]

In [ ]:
num_iterations = 10
results, features, dims, scores = run_pso_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations:   0%|          | 0/10 [00:00<?, ?iteration/s]